## an example with word2vec based on gensim
This notebook introduces the example from the gensim word2vec source code.

**gensim version 4.0.1**

### Usage
- 输入: 句子分词的结果
- 输出: 对应每个词的向量表示

In [2]:
from gensim.models import Word2Vec

# dataset
common_texts = [
    ['human', 'interface', 'computer'],
    ['survey', 'user', 'computer', 'system', 'response', 'time'],
    ['eps', 'user', 'interface', 'system'],
    ['system', 'human', 'system', 'eps'],
    ['user', 'response', 'time'],
    ['trees'],
    ['graph', 'trees'],
    ['graph', 'minors', 'trees'],
    ['graph', 'minors', 'survey']
]

# construct model
model = Word2Vec(sentences=common_texts, vector_size=100, window=5, min_count=1, workers=4)

# obtain word vector example
vector = model.wv['computer']  # get numpy vector of a word
sims = model.wv.most_similar('computer', topn=10)  # get other similar words

print("[computer] vector: ", vector)
print('\n')
print("similar words: ", sims)

[computer] vector:  [-0.00515774 -0.00667028 -0.0077791   0.00831315 -0.00198292 -0.00685696
 -0.0041556   0.00514562 -0.00286997 -0.00375075  0.0016219  -0.0027771
 -0.00158482  0.0010748  -0.00297881  0.00852176  0.00391207 -0.00996176
  0.00626142 -0.00675622  0.00076966  0.00440552 -0.00510486 -0.00211128
  0.00809783 -0.00424503 -0.00763848  0.00926061 -0.00215612 -0.00472081
  0.00857329  0.00428458  0.0043261   0.00928722 -0.00845554  0.00525685
  0.00203994  0.0041895   0.00169839  0.00446543  0.00448759  0.0061063
 -0.00320303 -0.00457706 -0.00042664  0.00253447 -0.00326412  0.00605948
  0.00415534  0.00776685  0.00257002  0.00811904 -0.00138761  0.00808028
  0.0037181  -0.00804967 -0.00393476 -0.0024726   0.00489447 -0.00087241
 -0.00283173  0.00783599  0.00932561 -0.0016154  -0.00516075 -0.00470313
 -0.00484746 -0.00960562  0.00137242 -0.00422615  0.00252744  0.00561612
 -0.00406709 -0.00959937  0.00154715 -0.00670207  0.0024959  -0.00378173
  0.00708048  0.00064041  0.00356

### Paramenters

Word2Vec类初始化函数
```python
def __init__(
        self, sentences=None, corpus_file=None, vector_size=100, alpha=0.025, window=5, min_count=5,
        max_vocab_size=None, sample=1e-3, seed=1, workers=3, min_alpha=0.0001,
        sg=0, hs=0, negative=5, ns_exponent=0.75, cbow_mean=1, hashfxn=hash, epochs=5, null_word=0,
        trim_rule=None, sorted_vocab=1, batch_words=MAX_WORDS_IN_BATCH, compute_loss=False, callbacks=(),
        comment=None, max_final_vocab=None,
    )
```

- sentences: 待输入分词结果
- corpus_file: 语料库文件路径, 格式要求详见 class:`~gensim.models.word2vec.LineSentence`
- vector_size: word vector的维度
- alpha: 学习率初始化
- window: 当前词与待预测词之间的最大距离
- min_count: 忽略频率小于此值的单词
- sg: (可选, 0或1) 0表示CBOW，1表示skip-gram
- hs: (可选, 0或1) 1表示模型训练时使用分层softmax, 0表示(且negative参数不为0),表示使用negative sampling
- negative: 负样本数量(noise word)
- nx_exponent: negative sampling分布指数, 值为1表示根据频率采样, 值为0表示对所有单词均等采样, 值为负值表示采样低频单词多于高频单词
- cbow_mean: 如果为0, 使用上下文词向量的和; 为1, 使用上下文词向量均值(仅适用cbow)
- min_alpha: 学习速率下降的最低值

### Source code

#### scan_vocab 单词初始化

```python
    def _scan_vocab(self, sentences, progress_per, trim_rule):
        sentence_no = -1
        total_words = 0
        min_reduce = 1
        vocab = defaultdict(int)
        checked_string_types = 0
        for sentence_no, sentence in enumerate(sentences):
            if not checked_string_types:
                if isinstance(sentence, str):
                    logger.warning(
                        "Each 'sentences' item should be a list of words (usually unicode strings). "
                        "First item here is instead plain %s.",
                        type(sentence),
                    )
                checked_string_types += 1
            if sentence_no % progress_per == 0:
                logger.info(
                    "PROGRESS: at sentence #%i, processed %i words, keeping %i word types",
                    sentence_no, total_words, len(vocab)
                )
            for word in sentence:
                vocab[word] += 1   # 记录每个词出现的次数
            total_words += len(sentence)  # 记录出现的单词总数

            if self.max_vocab_size and len(vocab) > self.max_vocab_size:
                utils.prune_vocab(vocab, min_reduce, trim_rule=trim_rule)
                min_reduce += 1

        corpus_count = sentence_no + 1
        self.raw_vocab = vocab  # 字典形式, 记录每个词出现的次数
        return total_words, corpus_count  # total words单词总数, corpus_count 句子数
```

- 输入: sentences分词结果
- 输出: total words单词总数, corpus_count文档数(sentence_length + 1)

#### prepare_vocab 采样控制

##### 函数头
```python
    def prepare_vocab(
            self, update=False, keep_raw_vocab=False, trim_rule=None,
            min_count=None, sample=None, dry_run=False,
        )
```
parameters:
- min_count: 忽略频率小于此值的单词
- sample: 控制more-frequent words的下采样

##### 加载新的词汇表
```python
       if not update:
            logger.info("Creating a fresh vocabulary")
            retain_total, retain_words = 0, []
            # Discard words less-frequent than min_count
            if not dry_run:
                self.wv.index_to_key = []
                # make stored settings match these applied settings
                self.min_count = min_count
                self.sample = sample
                self.wv.key_to_index = {}

            for word, v in self.raw_vocab.items():
                if keep_vocab_item(word, v, self.effective_min_count, trim_rule=trim_rule):
                    retain_words.append(word)
                    retain_total += v
                    if not dry_run:
                        self.wv.key_to_index[word] = len(self.wv.index_to_key)
                        self.wv.index_to_key.append(word)
                else:
                    drop_unique += 1
                    drop_total += v
```
- self.rae_vocab为**scan_vocab**中保留的单词表dict
- keep_vocab_item()判断单词是否需要被忽略/丢弃

##### 利用新的词汇更新模型
```python
        else:
            logger.info("Updating model with new vocabulary")
            new_total = pre_exist_total = 0
            new_words = []
            pre_exist_words = []
            for word, v in self.raw_vocab.items():
                if keep_vocab_item(word, v, self.effective_min_count, trim_rule=trim_rule):
                    if self.wv.has_index_for(word):
                        pre_exist_words.append(word)
                        pre_exist_total += v
                        if not dry_run:
                            pass
                    else:
                        new_words.append(word)
                        new_total += v
                        if not dry_run:
                            self.wv.key_to_index[word] = len(self.wv.index_to_key)
                            self.wv.index_to_key.append(word)
                else:
                    drop_unique += 1
                    drop_total += v
```
- 同上, 最终保留drop_unique(丢弃的单词数,不重复), drop_total(丢弃的单词总数)结果以及retain_words, retain_total

##### 计算采样阈值
```python
# Precalculate each vocabulary item's threshold for sampling
        if not sample:
            # no words downsampled
            threshold_count = retain_total
        elif sample < 1.0:
            # traditional meaning: set parameter as proportion of total
            threshold_count = sample * retain_total
        else:
            # new shorthand: sample >= 1 means downsample all words with higher count than sample
            threshold_count = int(sample * (3 + np.sqrt(5)) / 2)

        downsample_total, downsample_unique = 0, 0
        for w in retain_words:
            v = self.raw_vocab[w]
            word_probability = (np.sqrt(v / threshold_count) + 1) * (threshold_count / v)
            if word_probability < 1.0:
                downsample_unique += 1
                downsample_total += word_probability * v
            else:
                word_probability = 1.0
                downsample_total += v
            if not dry_run:
                self.wv.set_vecattr(w, 'sample_int', np.uint32(word_probability * (2**32 - 1)))
```
- 采样公式计算: `word_probability = (np.sqrt(v / threshold_count) + 1) * (threshold_count / v)`,其中threshold表示该词频数*采样率

##### 根据最终词汇表选择建立表格make_cum_table和构建二叉树create_binary_tree
```python
    # return from each step: words-affected, resulting-corpus-size, extra memory estimates
        if self.sorted_vocab and not update:
            self.wv.sort_by_descending_frequency()

        if self.hs:
            # add info about each word's Huffman encoding
            self.create_binary_tree()
        if self.negative:
            # build the table for drawing random words (for negative sampling)
            self.make_cum_table()
```
- hs和negative分别对应两种构建词表方法

#### create_binary_tree() 构建二叉树

```python
    # constructing a huffman tree from %i words
    heap = _build_heap(wv)
    if not heap:
        #
        # TODO: how can we end up with an empty heap?
        #
        logger.info("built huffman tree with maximum node depth 0")
        return

    # recurse over the tree, assigning a binary code to each vocabulary word
    max_depth = 0
    stack = [(heap[0], [], [])]
    while stack:
        node, codes, points = stack.pop()
        if node[1] < len(wv):  # node[1] = index
            # leaf node => store its path from the root
            k = node[1]
            wv.set_vecattr(k, 'code', codes)
            wv.set_vecattr(k, 'point', points)
            # node.code, node.point = codes, points
            max_depth = max(len(codes), max_depth)
        else:
            # inner node => continue recursion
            points = np.array(list(points) + [node.index - len(wv)], dtype=np.uint32)
            stack.append((node.left, np.array(list(codes) + [0], dtype=np.uint8), points))
            stack.append((node.right, np.array(list(codes) + [1], dtype=np.uint8), points))
```
- 哈夫曼树, 带权路径长度最短的树
- 保证词频高的单词路径短,词频相对较低的单词的路径长,可以减少计算量

#### make_cum_table() 负采样

```python
    def make_cum_table(self, domain=2**31 - 1):
        vocab_size = len(self.wv.index_to_key)
        self.cum_table = np.zeros(vocab_size, dtype=np.uint32)
        # compute sum of all power (Z in paper)
        train_words_pow = 0.0
        for word_index in range(vocab_size):
            count = self.wv.get_vecattr(word_index, 'count')
            train_words_pow += count**self.ns_exponent
        cumulative = 0.0
        for word_index in range(vocab_size):
            count = self.wv.get_vecattr(word_index, 'count')
            cumulative += count**self.ns_exponent
            self.cum_table[word_index] = round(cumulative / train_words_pow * domain)
        if len(self.cum_table) > 0:
            assert self.cum_table[-1] == domain
```
- 负采样原理:
    - 将长度为1的线段分为$V$份($V$为词汇表大小), 每份对应词汇表中的一个词(此时,词汇表中的词按频次排序)
    - 高频词对应的线段长, 低频次对应的线段短(每个词$w$的线段长度计算公式如下)

        $$len(w)=\frac{count(w)^{ns\_exponent}}{\sum_{u \in vocab}count(u)^{ns\_exponent}}$$

    - 采样前, 将这段长度为1的线段分成$M$等份($M>>N$), 并与之前的线段进行映射(可以保证没歌词对应的线段都会划分为对应的小块)
    - 采样时, 从$M$个位置中采样出 neg 个位置,此时采样到的每个位置对应线段所属的词就是负样本词
- parameters
    - domain 表示均分的份数,即对应$M$
    - nx_exponent 表示指数幂, 默认为0.75
    - cumulative 表示公式中的分子
    - train_words_pow 表示公式中的分母
- 代码实际计算的是在这条线段中,所占据部分的右端index, 因此计算公式为`round(cumulative / train_words_pow * domain)`
- 输出负采样所需词对应占比表

#### train_batch_sg() skip_gram模型 & train_batch_cbow CBOW模型
gensim使用的C版本的模型
```python
from gensim.models.word2vec_inner import (  # noqa: F401
        train_batch_sg,
        train_batch_cbow,
        score_sentence_sg,
        score_sentence_cbow,
        MAX_WORDS_IN_BATCH,
        FAST_VERSION,
    )
```
